<a href="https://colab.research.google.com/github/AjeetSingh02/Notebooks/blob/master/profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install pandas_profiling==2.3.0

In [0]:
import pandas as pd
import time
import pandas_profiling

df = pd.read_csv("/content/sample_data/california_housing_train.csv")

start = time.time()

profile = pandas_profiling.ProfileReport(df)
parent_dict = profile.get_description()

end = time.time()
print(end - start)

print(parent_dict.keys())
print(len(parent_dict["table"].keys()))
print(parent_dict["correlations"].keys())
print(len(parent_dict["variables"]["households"]))

31.455469608306885
dict_keys(['table', 'variables', 'correlations', 'missing', 'messages'])
22
dict_keys(['pearson', 'spearman', 'kendall', 'phi_k'])
47


In [0]:
parent_dict['variables']["households"].keys()

dict_keys(['value_counts_with_nan', 'value_counts_without_nan', 'distinct_count_with_nan', 'distinct_count_without_nan', 'type', 'count', 'distinct_count', 'p_missing', 'n_missing', 'p_infinite', 'n_infinite', 'is_unique', 'mode', 'p_unique', 'memorysize', 'mean', 'std', 'variance', 'min', 'max', 'kurtosis', 'skewness', 'sum', 'mad', 'n_zeros', 'histogramdata', 'range', '5%', '25%', '50%', '75%', '95%', 'iqr', 'cv', 'p_zeros', 'histogram_bins', 'histogram_bins_bayesian_blocks', 'varname', 'varid', 'row_classes', 'histogram', 'mini_histogram', 'histogram_bayesian_blocks', 'freqtable', 'firstn_expanded', 'lastn_expanded', 'sections'])

In [0]:
!pip install simplejson

     |████████████████████████████████| 81kB 2.9MB/s 
  Created wheel for simplejson: filename=simplejson-3.16.0-cp36-cp36m-linux_x86_64.whl size=114021 sha256=13caf7847ac53a63af1f7bb2828ba333a06cb16f23aa1d58ba21d33e4dfd6c45
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
Successfully built simplejson


In [0]:
# !pip show pandas_profiling

https://stackoverflow.com/questions/1773805/how-can-i-parse-a-yaml-file-in-python

https://stackoverflow.com/questions/12470665/how-can-i-write-data-in-yaml-format-in-a-file


In [0]:
!pip show pandas_profiling

Name: pandas-profiling
Version: 2.3.0
Summary: Generate profile report for pandas DataFrame
Home-page: https://github.com/pandas-profiling/pandas-profiling
Author: Jos Polfliet, Simon Brugman
Author-email: pandasprofiling@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: astropy, jinja2, phik, pandas, htmlmin, missingno, confuse, matplotlib
Required-by: 


In [0]:
#Variable.S_TYPE_CORR
#it means that this variable is highly correlated with some variable

#the name of the variable is given by "correlation_var"
#and the correlation is given by "correlation"

In [0]:
import pandas_profiling
import pandas as pd
import numpy as np
import simplejson
import yaml

#function to convert np.int64 into integer
def convert(o):
    if isinstance(o, np.int64): 
        return int(o) 
    elif isinstance(o, np.bool_):
        if o == True:
            return True
        return False

def save_json(csv_file, output_file_path, yaml_file):

    df = pd.read_csv(csv_file)
    profile = pandas_profiling.ProfileReport(df)

    parent_dict = profile.get_description()

    data = {}
    data["overview"] = parent_dict["table"]
    variables = parent_dict["variables"]

    with open(yaml_file, 'r') as stream:
        try:
            possible_keys = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            return f"following error occured while loading yaml file: {exc}"

    data["variables general info"] = {}
    data["variables general info"]["header"] = possible_keys


    data["variables frequency info"] = {}
    data["variables frequency info"]["value_counts_with_nan"] = {}
    data["variables frequency info"]["value_counts_without_nan"] = {}

    #For info of each variable
    for variable, dict_ in variables.items():
        temp_list = []
        for key in possible_keys:
            if key in dict_:
                if key == "type":
                    temp_list.append(dict_[key].value)
                else:  
                    temp_list.append(dict_[key])
            else:
                temp_list.append("None")
        data["variables general info"][variable] = temp_list

        #for frequency
        with_nan = dict_["value_counts_with_nan"]  
        without_nan = dict_["value_counts_without_nan"]

        data["variables frequency info"]["value_counts_with_nan"][variable] = [list(with_nan.index), list(with_nan.values)]
        data["variables frequency info"]["value_counts_without_nan"][variable] = [list(without_nan.index), list(without_nan.values)]

    try:
        correlations = parent_dict["correlations"]

        data["correlations"] = {}

        for corr, df_ in correlations.items():
            df_.fillna("NAN", inplace=True)
            data["correlations"][corr] = [list(df_.index), [list(val) for val in df_.values]]
            
    except:
        print("No correlation generated")

    #Saving at output_path
    with open(f"{output_file_path}output.json", "w") as write_file:
        simplejson.dump(data, write_file, default=convert, ignore_nan=True)  
    
    return data

In [0]:
csv_file = "/content/saa.csv"
output_file_path = ""
yaml_file = "/content/possible_keys.yml"
data = save_json(csv_file, output_file_path, yaml_file)

In [0]:
data

{'correlations': {'cramers': [['col4', 'col5'], [[1.0, 'NAN'], ['NAN', 1.0]]],
  'kendall': [['col1', 'col2', 'col3', 'col4'],
   [[1.0, 0.11870742829929376, 0.262431940540739, 'NAN'],
    [0.11870742829929376, 1.0, -0.17568209223157663, 'NAN'],
    [0.262431940540739, -0.17568209223157663, 1.0, 'NAN'],
    ['NAN', 'NAN', 'NAN', 1.0]]],
  'pearson': [['col1', 'col2', 'col3', 'col4'],
   [[1.0, 0.20802806985681668, 0.240004285829085, 'NAN'],
    [0.20802806985681668, 1.0, -0.3270052645568849, 'NAN'],
    [0.240004285829085, -0.3270052645568849, 1.0, 'NAN'],
    ['NAN', 'NAN', 'NAN', 'NAN']]],
  'phi_k': [['col1', 'col2', 'col3', 'col5'],
   [[1.0, 0.7061917956172633, 0.0, 1.0],
    [0.7061917956172633, 1.0, 0.7953682920810697, 1.0],
    [0.0, 0.7953682920810697, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0]]],
  'recoded': [['col4', 'col5'], [[1.0, 0.0], [0.0, 1.0]]],
  'spearman': [['col1', 'col2', 'col3', 'col4'],
   [[1.0, 0.0905895853426587, 0.3060730460762072, 'NAN'],
    [0.0905895853426587

In [0]:
possible_keys = ['distinct_count_with_nan', 'distinct_count_without_nan', 'type', 'count', 'distinct_count', 'p_missing', 'n_missing', 'p_infinite', 'n_infinite', 
            'is_unique', 'mode', 'p_unique', 'memorysize', 'top', 'freq', 'max_length', 'mean_length', 'min_length', 'composition', 'date_warning', 'varname', 
            'varid', 'row_classes', 'mean', 'std', 'variance', 'min', 'max', 'kurtosis', 'skewness', 'sum', 'mad', 'n_zeros', 'range', 
            '5%', '25%', '50%', '75%', '95%', 'iqr', 'cv', 'p_zeros', 'correlation_var', 'correlation']

with open('possible_keys.yml', 'w') as outfile:
    yaml.dump(possible_keys, outfile, default_flow_style=False)

In [0]:
import pandas as pd
import pandas_profiling

In [0]:
d = pd.read_csv("/content/test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import time

start = time.time()

profile = pandas_profiling.ProfileReport(d)

parent_dict = profile.get_description()

end = time.time()
print(end - start)

380.8393883705139


In [0]:
df = pd.read_csv("/content/sample_data/california_housing_test.csv")

In [0]:
profile = pandas_profiling.ProfileReport(df)

parent_dict = profile.get_description()

In [0]:
[list(without_nan.index), list(without_nan.values)]

In [0]:
print(index)
print(values)

[375.0, 273.0, 614.0, 340.0, 363.0, 429.0, 456.0, 335.0, 239.0, 287.0, 464.0, 301.0, 341.0, 275.0, 366.0, 389.0, 347.0, 279.0, 249.0, 229.0, 499.0, 290.0, 260.0, 228.0, 433.0, 334.0, 267.0, 406.0, 297.0, 278.0, 443.0, 581.0, 352.0, 507.0, 484.0, 577.0, 470.0, 417.0, 284.0, 270.0, 243.0, 497.0, 199.0, 274.0, 331.0, 314.0, 430.0, 221.0, 298.0, 795.0, 219.0, 246.0, 460.0, 396.0, 576.0, 292.0, 446.0, 327.0, 524.0, 504.0, 374.0, 503.0, 338.0, 306.0, 435.0, 593.0, 398.0, 355.0, 266.0, 364.0, 481.0, 242.0, 530.0, 467.0, 342.0, 261.0, 551.0, 447.0, 365.0, 227.0, 362.0, 311.0, 174.0, 288.0, 294.0, 356.0, 391.0, 357.0, 244.0, 367.0, 211.0, 231.0, 236.0, 285.0, 190.0, 472.0, 324.0, 532.0, 557.0, 606.0, 282.0, 394.0, 271.0, 412.0, 277.0, 276.0, 516.0, 251.0, 360.0, 329.0, 450.0, 351.0, 247.0, 215.0, 319.0, 418.0, 369.0, 397.0, 265.0, 318.0, 235.0, 286.0, 563.0, 631.0, 545.0, 377.0, 419.0, 485.0, 269.0, 431.0, 214.0, 252.0, 408.0, 425.0, 445.0, 376.0, 500.0, 136.0, 264.0, 434.0, 226.0, 257.0, 262.0

In [0]:
index

[375.0, 273.0, 614.0, 340.0, 363.0, 122.0, 1672.0, 951.0, 55.0, 11.0]

In [0]:
values

[12, 12, 12, 11, 11, 1, 1, 1, 1, 1]